[![Lab Documentation and Solutions](https://img.shields.io/badge/Lab%20Documentation%20and%20Solutions-purple)](https://mongodb-developer.github.io/vector-search-lab/)


In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AnyMessage
from langgraph.runtime import get_runtime
from dataclasses import dataclass
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel


class AgentResponse(BaseModel):
    # ensure strict compliance with the schema
    weather_conditions: str
    temperature: float
    address: str


@dataclass
class ContextSchema:
    user_name: str


# Load environment variables from .env file
load_dotenv()


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


def get_address(user: str) -> str:
    """Get users address"""
    return f"{user} lives at London, Elephant and Castle"


def prompt(state: AgentState) -> list[AnyMessage]:
    runtime = get_runtime(ContextSchema)
    system_msg = (
        f"You are a helpful assistant. Address the user as {runtime.context.user_name}."
    )
    return [{"role": "system", "content": system_msg}] + state["messages"]


agent = create_react_agent(
    model=init_chat_model(
        # model="openai:gpt-5",
        model="openai:gpt-4o-mini",
        # api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0.0,
        # max_tokens=1000,
    ),
    # the AI model will be queried to choose which tool to use based on the user's request
    tools=[get_weather, get_address],
    prompt=prompt,
    context_schema=ContextSchema,
    # the agent will save its state in memory
    checkpointer=InMemorySaver(),
    # checkpointer=MongoDBSaver(
    #     client=get_client().client,
    #     checkpointCollectionName="agent_state",
    #     db_name="playground"
    # ),
    # Structured output requires an additional call to the LLM to format the response according to the schema.
    response_format=AgentResponse,
)


In [ ]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf?"}]},
    context=ContextSchema(user_name="Ricardo Villanueva"),
    config={"thread_id": "123"},
)

In [ ]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "Where does she live?"}]},
    context=ContextSchema(user_name="Ricardo Villanueva"),
    config={"thread_id": "123"},
)

In [ ]:
# Run the agent
agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "Where does she live, and is it raining there?"}
        ]
    },
    context=ContextSchema(user_name="Ricardo Villanueva"),
    config={"thread_id": "123"},
)

In [ ]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "What did I say?"}]},
    context=ContextSchema(user_name="Ricardo Villanueva"),
    config={"thread_id": "123"},
)

In [ ]:
from src.examples.agent_with_mongodb_memory import invoke_agent

invoke_agent("What is the weather in London?", "John", "123")

In [ ]:
from src.examples.agent_with_mongodb_memory import invoke_agent

invoke_agent("What is the weather in Lisbon?", "John", "123")

/Users/ricardo.villanueva/miniconda3/envs/ai-agent-with-mongodb/lib/python3.13/site-packages/pymongo/common.py:844: UserWarning: Wire protocol compression with snappy is not available. You must install the python-snappy module for snappy support.
  value = validator(option, value)


{'messages': [HumanMessage(content='What is the weather in London?', additional_kwargs={}, response_metadata={}, id='343305b2-94a3-4eaf-958a-04c5502858f6'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w0M2DaVCXRGIqdR6xFtN5nPP', 'function': {'arguments': '{"city":"London","user":"John","context":{"user_name":"John"}}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 162, 'prompt_tokens': 178, 'total_tokens': 340, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3RkK0mIjOcfucuN9NMJll1YeszfD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--01154e9c-576c-439d-93fc-5b55f8af1b7f-0', tool_calls=[{'name': 'get_weather',